In [ ]:
!pip freeze > requirements.txt

In [ ]:
import zipfile
import os

# Caminho para o arquivo .zip
zip_path = '/content/microdados_enem_2020.zip'

# Diretório onde os arquivos serão extraídos
extract_path = 'extraidos'

# Descompactar o arquivo
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Listar arquivos extraídos
print("Arquivos extraídos:", os.listdir(extract_path))


Arquivos extraídos: ['INPUTS', 'LEIA-ME E DOCUMENTOS TÉCNICOS', 'DADOS', 'DICION╡RIO', 'PROVAS E GABARITOS']


## UTILIZANDO MICRODADOS_ENEM_2020

In [ ]:
import sqlite3
import pandas as pd

# Caminho para o arquivo CSV
csv_path = '/content/extraidos/DADOS/MICRODADOS_ENEM_2020.csv'

# Nome do banco de dados SQLite
db_path = 'enem_2020.db'

# Nome da tabela no banco
table_name = 'microdados_enem'

# Ler o CSV em um DataFrame
print("Lendo o arquivo CSV...")
df = pd.read_csv(csv_path, sep=';', encoding='latin1')  # Ajuste 'sep' e 'encoding' se necessário

# Conectar ao banco de dados SQLite (será criado se não existir)
print("Criando o banco de dados SQLite...")
conn = sqlite3.connect(db_path)

# Escrever os dados do DataFrame na tabela SQLite
print(f"Inserindo os dados na tabela '{table_name}'...")
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Fechar a conexão
conn.close()

print(f"Banco de dados '{db_path}' criado com sucesso!")

Lendo o arquivo CSV...
Criando o banco de dados SQLite...
Inserindo os dados na tabela 'microdados_enem'...
Banco de dados 'enem_2020.db' criado com sucesso!


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = '/content/enem_2020.db'

# Nome da tabela para consultar as variáveis
table_name = 'microdados_enem'

# Conectar ao banco de dados
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Executar o comando PRAGMA para obter informações sobre as colunas
query = f"PRAGMA table_info({table_name});"
cursor.execute(query)

# Buscar todas as colunas
columns_info = cursor.fetchall()

# Exibir informações das colunas
print(f"Colunas da tabela '{table_name}':")
for col in columns_info:
    print(f"- Nome: {col[1]}, Tipo: {col[2]}, Not Null: {bool(col[3])}, PK: {bool(col[5])}")

# Fechar a conexão
conn.close()


Colunas da tabela 'microdados_enem':
- Nome: NU_INSCRICAO, Tipo: INTEGER, Not Null: False, PK: False
- Nome: NU_ANO, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_FAIXA_ETARIA, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_SEXO, Tipo: TEXT, Not Null: False, PK: False
- Nome: TP_ESTADO_CIVIL, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_COR_RACA, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_NACIONALIDADE, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_ST_CONCLUSAO, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_ANO_CONCLUIU, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_ESCOLA, Tipo: INTEGER, Not Null: False, PK: False
- Nome: TP_ENSINO, Tipo: REAL, Not Null: False, PK: False
- Nome: IN_TREINEIRO, Tipo: INTEGER, Not Null: False, PK: False
- Nome: CO_MUNICIPIO_ESC, Tipo: REAL, Not Null: False, PK: False
- Nome: NO_MUNICIPIO_ESC, Tipo: TEXT, Not Null: False, PK: False
- Nome: CO_UF_ESC, Tipo: REAL, Not Null: False, PK: False
- Nome: SG_UF_E

In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para encontrar a escola com a maior média de notas
query = """
SELECT
    CO_MUNICIPIO_ESC,
    AVG(NU_NOTA_CN) AS media_cn,
    AVG(NU_NOTA_CH) AS media_ch,
    AVG(NU_NOTA_LC) AS media_lc,
    AVG(NU_NOTA_MT) AS media_mt,
    (AVG(NU_NOTA_CN) + AVG(NU_NOTA_CH) + AVG(NU_NOTA_LC) + AVG(NU_NOTA_MT)) / 4 AS media_total
FROM
    microdados_enem
GROUP BY
    CO_MUNICIPIO_ESC
ORDER BY
    media_total DESC
LIMIT 1;
"""

# Executar a consulta
cursor.execute(query)

# Buscar o resultado
result = cursor.fetchone()

# Exibir o resultado
if result:
    print("Escola com a maior média de notas:")
    print(f"ID do Município: {result[0]}, Média CN: {result[1]}, Média CH: {result[2]}, Média LC: {result[3]}, Média MT: {result[4]}, Média Total: {result[5]}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Escola com a maior média de notas:
ID do Município: 4117297.0, Média CN: 586.1, Média CH: 779.6, Média LC: 674.6, Média MT: 823.2, Média Total: 715.875


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para encontrar o aluno com a maior média de notas
query = """
SELECT
    NU_INSCRICAO,
    (NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT) / 4.0 AS media_total
FROM
    microdados_enem
WHERE
    NU_NOTA_CN IS NOT NULL AND
    NU_NOTA_CH IS NOT NULL AND
    NU_NOTA_LC IS NOT NULL AND
    NU_NOTA_MT IS NOT NULL
ORDER BY
    media_total DESC
LIMIT 1;
"""

# Executar a consulta
cursor.execute(query)

# Buscar o resultado
result = cursor.fetchone()

# Exibir o resultado
if result:
    print("Aluno com a maior média de notas:")
    print(f"NU_INSCRICAO: {result[0]}, Média Total: {result[1]}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Aluno com a maior média de notas:
NU_INSCRICAO: 200001357436, Média Total: 830.95


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular a média geral das notas
query = """
SELECT
    AVG((NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT) / 4.0) AS media_geral
FROM
    microdados_enem
WHERE
    NU_NOTA_CN IS NOT NULL AND
    NU_NOTA_CH IS NOT NULL AND
    NU_NOTA_LC IS NOT NULL AND
    NU_NOTA_MT IS NOT NULL;
"""

# Executar a consulta
cursor.execute(query)

# Buscar o resultado
result = cursor.fetchone()

# Exibir o resultado
if result:
    print(f"Média geral das notas: {result[0]}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Média geral das notas: 512.8920297634561


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular a porcentagem de ausentes
query = """
SELECT
    (SUM(CASE
            WHEN TP_PRESENCA_CN = 0 THEN 1
            ELSE 0
         END) +
     SUM(CASE
            WHEN TP_PRESENCA_CH = 0 THEN 1
            ELSE 0
         END) +
     SUM(CASE
            WHEN TP_PRESENCA_LC = 0 THEN 1
            ELSE 0
         END) +
     SUM(CASE
            WHEN TP_PRESENCA_MT = 0 THEN 1
            ELSE 0
         END)) * 100.0 / COUNT(*) AS percentual_ausentes
FROM
    microdados_enem;
"""

# Executar a consulta
cursor.execute(query)

# Buscar o resultado
result = cursor.fetchone()

# Exibir o resultado
if result:
    print(f"Percentual de Ausentes: {result[0]:.2f}%")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Percentual de Ausentes: 214.72%


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para contar o número total de inscritos
query = """
SELECT COUNT(*) AS total_inscritos
FROM microdados_enem;
"""

# Executar a consulta
cursor.execute(query)

# Buscar o resultado
result = cursor.fetchone()

# Exibir o resultado
if result:
    print(f"Número total de Inscritos: {result[0]}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Número total de Inscritos: 5783109


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular a média por disciplina
query = """
SELECT
    AVG(NU_NOTA_CN) AS media_CN,
    AVG(NU_NOTA_CH) AS media_CH,
    AVG(NU_NOTA_LC) AS media_LC,
    AVG(NU_NOTA_MT) AS media_MT
FROM
    microdados_enem
WHERE
    NU_NOTA_CN IS NOT NULL AND
    NU_NOTA_CH IS NOT NULL AND
    NU_NOTA_LC IS NOT NULL AND
    NU_NOTA_MT IS NOT NULL;
"""

# Executar a consulta
cursor.execute(query)

# Buscar o resultado
result = cursor.fetchone()

# Exibir o resultado
if result:
    print(f"Média Ciências da Natureza: {result[0]:.2f}")
    print(f"Média Ciências Humanas: {result[1]:.2f}")
    print(f"Média Linguagens: {result[2]:.2f}")
    print(f"Média Matemática: {result[3]:.2f}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Média Ciências da Natureza: 490.53
Média Ciências Humanas: 514.29
Média Linguagens: 526.02
Média Matemática: 520.73


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular a média por sexo
query = """
SELECT
    TP_SEXO,
    AVG(NU_NOTA_CN) AS media_CN,
    AVG(NU_NOTA_CH) AS media_CH,
    AVG(NU_NOTA_LC) AS media_LC,
    AVG(NU_NOTA_MT) AS media_MT
FROM
    microdados_enem
WHERE
    NU_NOTA_CN IS NOT NULL AND
    NU_NOTA_CH IS NOT NULL AND
    NU_NOTA_LC IS NOT NULL AND
    NU_NOTA_MT IS NOT NULL
GROUP BY
    TP_SEXO;
"""

# Executar a consulta
cursor.execute(query)

# Buscar todos os resultados
results = cursor.fetchall()

# Exibir o resultado
if results:
    for row in results:
        sexo, media_CN, media_CH, media_LC, media_MT = row
        print(f"Sexo: {sexo}")
        print(f"  Média Ciências da Natureza: {media_CN:.2f}")
        print(f"  Média Ciências Humanas: {media_CH:.2f}")
        print(f"  Média Linguagens: {media_LC:.2f}")
        print(f"  Média Matemática: {media_MT:.2f}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Sexo: F
  Média Ciências da Natureza: 480.33
  Média Ciências Humanas: 505.58
  Média Linguagens: 523.72
  Média Matemática: 500.83
Sexo: M
  Média Ciências da Natureza: 506.12
  Média Ciências Humanas: 527.57
  Média Linguagens: 529.54
  Média Matemática: 551.11


In [ ]:
import sqlite3

# Caminho para o banco de dados
db_path = 'enem_2020.db'

# Conectar ao banco
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Consulta SQL para calcular a média por etnia
query = """
SELECT
    TP_COR_RACA,
    AVG(NU_NOTA_CN) AS media_CN,
    AVG(NU_NOTA_CH) AS media_CH,
    AVG(NU_NOTA_LC) AS media_LC,
    AVG(NU_NOTA_MT) AS media_MT
FROM
    microdados_enem
WHERE
    NU_NOTA_CN IS NOT NULL AND
    NU_NOTA_CH IS NOT NULL AND
    NU_NOTA_LC IS NOT NULL AND
    NU_NOTA_MT IS NOT NULL
GROUP BY
    TP_COR_RACA;
"""

# Executar a consulta
cursor.execute(query)

# Buscar todos os resultados
results = cursor.fetchall()

# Exibir o resultado
if results:
    for row in results:
        etnia, media_CN, media_CH, media_LC, media_MT = row
        print(f"Etnia: {etnia}")
        print(f"  Média Ciências da Natureza: {media_CN:.2f}")
        print(f"  Média Ciências Humanas: {media_CH:.2f}")
        print(f"  Média Linguagens: {media_LC:.2f}")
        print(f"  Média Matemática: {media_MT:.2f}")
else:
    print("Nenhum resultado encontrado.")

# Fechar a conexão
conn.close()


Etnia: 0
  Média Ciências da Natureza: 499.98
  Média Ciências Humanas: 525.75
  Média Linguagens: 531.74
  Média Matemática: 530.85
Etnia: 1
  Média Ciências da Natureza: 513.27
  Média Ciências Humanas: 541.21
  Média Linguagens: 547.46
  Média Matemática: 557.73
Etnia: 2
  Média Ciências da Natureza: 470.74
  Média Ciências Humanas: 493.33
  Média Linguagens: 511.79
  Média Matemática: 486.46
Etnia: 3
  Média Ciências da Natureza: 476.70
  Média Ciências Humanas: 497.55
  Média Linguagens: 512.23
  Média Matemática: 498.91
Etnia: 4
  Média Ciências da Natureza: 491.12
  Média Ciências Humanas: 509.03
  Média Linguagens: 522.10
  Média Matemática: 522.65
Etnia: 5
  Média Ciências da Natureza: 453.44
  Média Ciências Humanas: 468.55
  Média Linguagens: 483.52
  Média Matemática: 463.13
